In [1]:
from imports import *

activation_embeds = {
        "layer 0": [],
        "layer 1": [],
        "layer 2": [],
        "layer 3": [],
        "layer 4": [],
        "layer 5": []
    }

/Users/maheepchaudhary/miniforge3/envs/transit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for k in activation_embeds.keys():
    print(k)

layer 0
layer 1
layer 2
layer 3
layer 4
layer 5


In [18]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

x = np.array([[2,3,43,44,5]])
y = np.array([[2,3,43,44,5]])
distance, path = fastdtw(x, y, dist=euclidean)
print(distance)

0.0


In [2]:
_, path = fastdtw(reference, flattened_act, dist=euclidean)

NameError: name 'reference' is not defined

In [22]:
from imports import *
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean


def average_activation_for_layer(model_name):
    datasets = ["tinystories", "summarisation", "alpaca"]

    if model_name == "Pythia14m" or model_name == "Pythia70m":
        model_layer = 6
        activations = {f"layer {i}": [] for i in range(model_layer)}
        
    elif model_name == "Pythia160m":
        model_layer = 12
        activations = {f"layer {i}": [] for i in range(model_layer)}
        
    elif model_name == "Pythia410m":
        model_layer = 24
        activations = {f"layer {i}": [] for i in range(model_layer)}
    
    elif model_name == "Pythia1b":
        model_layer = 16
        activations = {f"layer {i}": [] for i in range(model_layer)}
        
    # Collect activations from datasets
    for dataset in datasets:
        file_path = f"data/{dataset}/{model_name}/activation_resid.pkl"
        if os.path.exists(file_path):
            with open(file_path, "rb") as f:
                data = pickle.load(f)
                for count, act_data in enumerate(data):
                    activations[f"layer {count}"].append(act_data)

    avg_activations = {}
    
    # Align activations using DTW and compute the average
    for layer, acts in activations.items():
        if acts:
            reference = np.array([acts[0]]).reshape(-1,1)  # Flatten the reference activation
            aligned_acts = []
            for act in acts:
                flattened_act = np.array([act]).reshape(-1,1)  # Flatten each activation before DTW
                _, path = fastdtw(reference, flattened_act, dist=euclidean)
                
                # Align activations based on DTW path
                aligned_acts.append([flattened_act[i] for i, _ in path])
                
            # Compute the mean of the aligned activations
            avg_activations[layer] = np.mean(aligned_acts, axis=0)
        else:
            raise FileNotFoundError(f"No activation data found for the specified model and layer {layer}.")

    return avg_activations

# Call the function
avg_activation = average_activation_for_layer("Pythia70m")


IndexError: index 10 is out of bounds for axis 0 with size 10